In [3]:
!pip install wget
!pip install pm4py
!pip install processtransformer==0.1.3 

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=59b2751ec514f96d9792a0597863e4b8b7a1b925dfcf99c0928b02f8246e16fa
  Stored in directory: c:\users\mahdi\appdata\local\pip\cache\wheels\04\5f\3e\46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [4]:
import os
import wget
import argparse
import pm4py
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics 

from processtransformer import constants
from processtransformer.models import transformer
from processtransformer.data.loader import LogsDataLoader
from processtransformer.data.processor import LogsDataProcessor


In [5]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
data_dir = "./datasets/"
if not os.path.exists(data_dir): 
  os.mkdir(data_dir)
_ = wget.download("https://data.4tu.nl/file/34c3f44b-3101-4ea9-8281-e38905c68b8d/f3aec4f7-d52c-4217-82f4-57d719a8298c")


input_file_path= wget.download("https://data.4tu.nl/file/34c3f44b-3101-4ea9-8281-e38905c68b8d/f3aec4f7-d52c-4217-82f4-57d719a8298c")
output_file_path='./datasets/BPIC17.csv'

#Write to Pandas Dataframe
log = pm4py.read_xes(input_file_path) #Input Filename
df = pm4py.convert_to_dataframe(log)
df.to_csv(output_file_path)

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

In [9]:
data_processor = LogsDataProcessor(name='BPIC17', filepath="datasets/BPIC17.csv",  
                                    columns = ["case:concept:name", "concept:name", "time:timestamp"], #specify the columns name containing case_id, activity name and timestamp 
                                    dir_path='datasets', pool = 4)
data_processor.process_logs(task=constants.Task.NEXT_ACTIVITY, sort_temporally= False)

In [10]:
# Load data
data_loader = LogsDataLoader(name = 'BPIC17')

(train_df, test_df, x_word_dict, y_word_dict, max_case_length, 
    vocab_size, num_output) = data_loader.load_data(constants.Task.NEXT_ACTIVITY)

# Prepare training examples for next activity prediction task
train_token_x, train_token_y = data_loader.prepare_data_next_activity(train_df, 
    x_word_dict, y_word_dict, max_case_length)

In [11]:
learning_rate = 0.001
batch_size = 12
epochs = 10

In [12]:
# Create and train a transformer model
transformer_model = transformer.get_next_activity_model(
    max_case_length=max_case_length, 
    vocab_size=vocab_size,
    output_dim=num_output)

transformer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

transformer_model.fit(train_token_x, train_token_y, 
    epochs=epochs, batch_size=batch_size)

Epoch 1/10
78000/78000 [==============================] - 5445s 70ms/step - loss: 0.4530 - sparse_categorical_accuracy: 0.8471
Epoch 2/10
78000/78000 [==============================] - 5442s 70ms/step - loss: 0.3707 - sparse_categorical_accuracy: 0.8697
Epoch 3/10
78000/78000 [==============================] - 5425s 70ms/step - loss: 0.3634 - sparse_categorical_accuracy: 0.8705
Epoch 4/10
78000/78000 [==============================] - 5639s 72ms/step - loss: 0.3594 - sparse_categorical_accuracy: 0.8713
Epoch 5/10
78000/78000 [==============================] - 6683s 86ms/step - loss: 0.3571 - sparse_categorical_accuracy: 0.8716
Epoch 6/10
78000/78000 [==============================] - 8154s 105ms/step - loss: 0.3566 - sparse_categorical_accuracy: 0.8716
Epoch 7/10
78000/78000 [==============================] - 7717s 99ms/step - loss: 0.3559 - sparse_categorical_accuracy: 0.8719
Epoch 8/10
78000/78000 [==============================] - 7036s 90ms/step - loss: 0.3557 - sparse_categorical_

In [13]:
# Evaluate over all the prefixes (k) and save the results
k, accuracies,fscores, precisions, recalls = [],[],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, test_token_y = data_loader.prepare_data_next_activity(test_data_subset, 
            x_word_dict, y_word_dict, max_case_length)   
        y_pred = np.argmax(transformer_model.predict(test_token_x), axis=1)
        accuracy = metrics.accuracy_score(test_token_y, y_pred)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(
            test_token_y, y_pred, average="weighted")
        k.append(i)
        accuracies.append(accuracy)
        fscores.append(fscore)
        precisions.append(precision)
        recalls.append(recall)

k.append(i + 1)
accuracies.append(np.mean(accuracy))
fscores.append(np.mean(fscores))
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))

1/1 [==============================] - 0s 55ms/step


In [14]:
print('Average accuracy across all prefixes:', np.mean(accuracies))
print('Average f-score across all prefixes:', np.mean(fscores))
print('Average precision across all prefixes:', np.mean(precisions))
print('Average recall across all prefixes:', np.mean(recalls))

Average accuracy across all prefixes: 0.8426708183994187
Average f-score across all prefixes: 0.8174516907598661
Average precision across all prefixes: 0.810716546424318
Average recall across all prefixes: 0.841607783388604


In [15]:
data_processor = LogsDataProcessor(name='BPIC17', filepath="datasets/BPIC17.csv",  
                                    columns = ["case:concept:name", "concept:name", "time:timestamp"],  #specify the columns name containing case_id, activity name and timestamp 
                                    dir_path='datasets', pool = 4)
data_processor.process_logs(task=constants.Task.NEXT_TIME, sort_temporally= False)

In [16]:
# Load data
data_loader = LogsDataLoader(name = 'BPIC17')

(train_df, test_df, x_word_dict, y_word_dict, max_case_length, 
    vocab_size, num_output) = data_loader.load_data(constants.Task.NEXT_TIME)

# Prepare training examples for next time prediction task
(train_token_x, train_time_x, train_token_y, time_scaler, y_scaler) = \
                                    data_loader.prepare_data_next_time(train_df, x_word_dict, max_case_length)

In [17]:
learning_rate = 0.001
batch_size = 12
epochs = 1

In [18]:
# Create and train a transformer model
transformer_model = transformer.get_next_time_model(
        max_case_length=max_case_length, 
        vocab_size=vocab_size)

transformer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss=tf.keras.losses.LogCosh())

transformer_model.fit([train_token_x, train_time_x], train_token_y, 
        epochs=epochs, batch_size=batch_size, verbose=2)

80101/80101 - 8046s - loss: 0.1079 - 8046s/epoch - 100ms/step


In [19]:
# Evaluate over all the prefixes (k) and save the results
k, maes, mses, rmses = [],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, test_time_x, test_y, _, _ = data_loader.prepare_data_next_time(
            test_data_subset, x_word_dict, max_case_length, time_scaler, y_scaler, False)   

        y_pred = transformer_model.predict([test_token_x, test_time_x])
        _test_y = y_scaler.inverse_transform(test_y)
        _y_pred = y_scaler.inverse_transform(y_pred)

        k.append(i)
        maes.append(metrics.mean_absolute_error(_test_y, _y_pred))
        mses.append(metrics.mean_squared_error(_test_y, _y_pred))
        rmses.append(np.sqrt(metrics.mean_squared_error(_test_y, _y_pred)))

k.append(i + 1)
maes.append(np.mean(maes))
mses.append(np.mean(mses))
rmses.append(np.mean(rmses))  

1/1 [==============================] - 0s 31ms/step


In [20]:
print('Average MAE across all prefixes:', np.mean(maes))
print('Average MSE across all prefixes:', np.mean(mses))
print('Average RMSE across all prefixes:', np.mean(rmses))

Average MAE across all prefixes: 0.3276172
Average MSE across all prefixes: 2.1393125
Average RMSE across all prefixes: 1.1289705


In [21]:
data_processor = LogsDataProcessor(name='BPIC17', filepath="datasets/BPIC17.csv",  
                                    columns = ["case:concept:name", "concept:name", "time:timestamp"],  #specify the columns name containing case_id, activity name and timestamp 
                                    dir_path='datasets', pool = 4)
data_processor.process_logs(task=constants.Task.REMAINING_TIME, sort_temporally= False)

In [22]:
# Load data
data_loader = LogsDataLoader(name = 'BPIC17')

(train_df, test_df, x_word_dict, y_word_dict, max_case_length, 
    vocab_size, num_output) = data_loader.load_data(constants.Task.REMAINING_TIME)

# Prepare training examples for next time prediction task
(train_token_x, train_time_x, 
    train_token_y, time_scaler, y_scaler) = data_loader.prepare_data_remaining_time(train_df, 
    x_word_dict, max_case_length)

In [23]:
learning_rate = 0.001
batch_size = 12
epochs = 1

In [24]:
# Create and train a transformer model
transformer_model = transformer.get_remaining_time_model(
    max_case_length=max_case_length, 
    vocab_size=vocab_size)

transformer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.LogCosh())

transformer_model.fit([train_token_x, train_time_x], train_token_y, 
        epochs=epochs, batch_size=batch_size, verbose=2)

80101/80101 - 7172s - loss: 0.2254 - 7172s/epoch - 90ms/step


In [25]:
# Evaluate over all the prefixes (k) and save the results
k, maes, mses, rmses = [],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, test_time_x, test_y, _, _ = data_loader.prepare_data_remaining_time(
            test_data_subset, x_word_dict, max_case_length, time_scaler, y_scaler, False)   

        y_pred = transformer_model.predict([test_token_x, test_time_x])
        _test_y = y_scaler.inverse_transform(test_y)
        _y_pred = y_scaler.inverse_transform(y_pred)

        k.append(i)
        maes.append(metrics.mean_absolute_error(_test_y, _y_pred))
        mses.append(metrics.mean_squared_error(_test_y, _y_pred))
        rmses.append(np.sqrt(metrics.mean_squared_error(_test_y, _y_pred)))

k.append(i + 1)
maes.append(np.mean(maes))
mses.append(np.mean(mses))
rmses.append(np.mean(rmses))  

1/1 [==============================] - 0s 35ms/step


In [26]:
print('Average MAE across all prefixes:', np.mean(maes))
print('Average MSE across all prefixes:', np.mean(mses))
print('Average RMSE across all prefixes:', np.mean(rmses))

Average MAE across all prefixes: 5.2377768
Average MSE across all prefixes: 54.289852
Average RMSE across all prefixes: 7.0011034
